# Sesión 2 — SOLUCIÓN (Solo instructor)

Solución de referencia para el instructor.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv('../data/telco_churn.csv')
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')
df.head()

## Actividad 1 — Target formal

- **Target:** y_i = 1[cliente i cancela en [t+1, t+3]]
- **Horizonte:** n=3 meses
- **Ventana:** Datos hasta t (exclusivo)
- **Unidad:** Cliente
- **Tipo:** Clasificación binaria

## Actividad 2 — Selección de variables

| Variable | Categoría | Justificación |
|----------|-----------|---------------|
| tenure | Válida | Antigüedad en t |
| MonthlyCharges | Válida | Cargo actual en t |
| TotalCharges | Leakage | Incluye meses futuros |
| Contract | Válida | Disponible en t |
| PaymentMethod | Válida | Disponible en t |
| InternetService | Válida | Disponible en t |
| customerID | Irrelevante | Identificador |
| Churn | Target | No predictora |
| ... | ... | ... |

## Actividad 3 — Validación empírica

In [ ]:
numeric_cols = ['tenure', 'MonthlyCharges', 'TotalCharges']
df_clean = df[numeric_cols].dropna()

corr_matrix = df_clean.corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, fmt='.2f')
plt.title('Matriz de correlación')
plt.show()

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

X = df_clean
X = (X - X.mean()) / X.std()  # Estandarizar para VIF
vif_data = pd.DataFrame()
vif_data['Variable'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
vif_data

In [ ]:
# Desbalanceo
df['Churn'].value_counts(normalize=True)
# ~73% No, ~27% Yes → desbalanceo moderado, considerar class_weight o sampling

**Decisiones:**
- VIF: TotalCharges y tenure correlacionados (tenure*MonthlyCharges≈TotalCharges). Eliminar TotalCharges por leakage y multicolinealidad.
- Encoding: One-hot para categóricas nominales.
- Normalización: Sí para logística/SVM.
- Sampling: class_weight='balanced' o SMOTE si desbalanceo severo.

## Actividad 4 — Simulación de fuga

- **Leakage:** Variable con info post-evento (TotalCharges, days_since_cancellation)
- **Inflación:** Accuracy/AUC altos en validación, colapso en producción
- **Detección:** Drift monitoring, comparar offline vs online
- **Prevención:** Punto de corte temporal estricto, features solo disponibles en t

## Ficha Técnica

| Campo | Contenido |
|-------|----------|
| Problema | Reducir churn en telecomunicaciones |
| KPI | Tasa churn, Score riesgo |
| Target | y=1[churn en t+1 a t+3] |
| Horizonte | 3 meses |
| Variables | tenure, MonthlyCharges, Contract, PaymentMethod, ... |
| Riesgos | Leakage, desbalanceo, drift |
| Modelo | Regresión logística / Random Forest |
| Validación | Train/test temporal 80/20 |